In [4]:
import requests
import time
import pandas as pd

URL = "https://www.vivino.com/api/explore/explore"  # o la URL que estés usando

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/123.0 Safari/537.36"
    )
}

# Parámetros base (los tuyos, sin la página)
PARAMS_BASE = {
    "currency_code": "EUR",
    "min_rating": 0,
    "order_by": "price",
    "order": "desc",
    "price_range_min": 0,
    "price_range_max": 500,
    "discount_prices": "false",
    "wine_style_ids[]": 180,   # Ribera del Duero tinto, por tu ejemplo
    "country_codes[]": "es",
    "wine_type_ids[]": 1,      # vino tinto
    "region_ids[]": 405        # Ribera del Duero
    # si usas el parámetro "e", lo añades aquí también
}

NUM_PAGES = 100

all_wines = []

for page in range(1, NUM_PAGES + 1):
    print(f"Scrapeando página {page}...")

    params = PARAMS_BASE.copy()
    params["page"] = page

    response = requests.get(URL, headers=HEADERS, params=params)
    response.raise_for_status()

    data = response.json()

    explore_data = data.get("explore_vintage") or data.get("exploreV2", {}).get("explore_vintage")
    records = explore_data.get("records", [])

    if not records:
        print("Sin más vinos en esta página, paro el bucle.")
        break

    for rec in records:
        vintage = rec.get("vintage", {})
        price_info = rec.get("price") or (rec.get("prices") or [None])[0]
        stats = vintage.get("statistics", {})

        wine_dict = {
            "pagina": page,  # nueva columna
            "vintage_id": vintage.get("id"),
            "nombre": vintage.get("name"),
            "seo_name": vintage.get("seo_name"),
            "year": vintage.get("year"),
            "rating": stats.get("ratings_average"),
            "n_ratings": stats.get("ratings_count"),
            "precio": price_info.get("amount") if price_info else None,
            "moneda": (price_info.get("currency") or {}).get("code") if price_info else None
        }

        all_wines.append(wine_dict)

    time.sleep(1)

# Generar DF y pasarlo a Excel
df_vinos = pd.DataFrame(all_wines)

print(df_vinos.shape)
print(df_vinos.head())

#Quitar duplicados columna vintage_id
df_vinos = df_vinos.drop_duplicates(subset=['vintage_id'])      
df_vinos.to_excel("vinos_vivino2.xlsx", index=False)


Scrapeando página 1...
Scrapeando página 2...
Scrapeando página 3...
Scrapeando página 4...
Scrapeando página 5...
Scrapeando página 6...
Scrapeando página 7...
Scrapeando página 8...
Scrapeando página 9...
Scrapeando página 10...
Scrapeando página 11...
Scrapeando página 12...
Scrapeando página 13...
Scrapeando página 14...
Scrapeando página 15...
Scrapeando página 16...
Scrapeando página 17...
Scrapeando página 18...
Scrapeando página 19...
Scrapeando página 20...
Scrapeando página 21...
Scrapeando página 22...
Scrapeando página 23...
Scrapeando página 24...
Scrapeando página 25...
Scrapeando página 26...
Scrapeando página 27...
Scrapeando página 28...
Scrapeando página 29...
Scrapeando página 30...
Scrapeando página 31...
Scrapeando página 32...
Scrapeando página 33...
Scrapeando página 34...
Scrapeando página 35...
Scrapeando página 36...
Scrapeando página 37...
Scrapeando página 38...
Scrapeando página 39...
Scrapeando página 40...
Scrapeando página 41...
Scrapeando página 42...
S